In [1]:
import os

path = os.path.join('..','..','..','datasets','cnn','stories')

##### 1. Loading and Segregating the CNN News Dataset

In [19]:
entries = os.listdir(path)
master_dict = {}
for file in entries:
    current = os.path.join(path,file)
    if os.path.isfile(current):
        story_list = []
        story_string = ""
        story_bool = True
        master_list = []
        with open(current, encoding="utf8") as file_nm:
            for line in file_nm:
                line = line.replace("\n", "")
                line = line.replace("\'", " ")
                if (line != "@highlight"):
                    if   story_bool:
                        if line == "":
                            continue
                        else:
                            story_string =story_string+" "+line
                    else:
                        if line == "":
                            continue
                        else:
                            story_list.append(line)
                else:
                    story_bool = False
            master_list.append(story_string)
            master_list.append(story_list)
            master_dict[file] = master_list


In [20]:
#for testing purposes
from itertools import islice
print(dict(islice(master_dict.items(), 5)))   # dict shortened
print(list(islice(master_dict.values(), 5)))  # values shortened

{'0001d1afc246a7964130f43ae940af6bc6c57f01.story': [' It s official: U.S. President Barack Obama wants lawmakers to weigh in on whether to use military force in Syria. Obama sent a letter to the heads of the House and Senate on Saturday night, hours after announcing that he believes military action against Syrian targets is the right step to take over the alleged use of chemical weapons. The proposed legislation from Obama asks Congress to approve the use of military force "to deter, disrupt, prevent and degrade the potential for future uses of chemical weapons or other weapons of mass destruction." It s a step that is set to turn an international crisis into a fierce domestic political battle. There are key questions looming over the debate: What did U.N. weapons inspectors find in Syria? What happens if Congress votes no? And how will the Syrian government react? In a televised address from the White House Rose Garden earlier Saturday, the president said he would take his case to Con

In [21]:
#for testing purposes - dummy example
for k, v in master_dict.items():
    if k=='0001d1afc246a7964130f43ae940af6bc6c57f01.story':
        print(k, v)

0001d1afc246a7964130f43ae940af6bc6c57f01.story [' It s official: U.S. President Barack Obama wants lawmakers to weigh in on whether to use military force in Syria. Obama sent a letter to the heads of the House and Senate on Saturday night, hours after announcing that he believes military action against Syrian targets is the right step to take over the alleged use of chemical weapons. The proposed legislation from Obama asks Congress to approve the use of military force "to deter, disrupt, prevent and degrade the potential for future uses of chemical weapons or other weapons of mass destruction." It s a step that is set to turn an international crisis into a fierce domestic political battle. There are key questions looming over the debate: What did U.N. weapons inspectors find in Syria? What happens if Congress votes no? And how will the Syrian government react? In a televised address from the White House Rose Garden earlier Saturday, the president said he would take his case to Congres

In [22]:
print(len(master_dict))

92579


In [23]:
#serialization
# file has been saved on client os
import pickle
with open('master_dict2.pkl', 'wb') as f:
    pickle.dump(master_dict, f)

In [1]:
import pickle
with open('master_dict2.pkl', 'rb') as f:
	master_dict = pickle.load(f)

##### 2. Extracting Summaries from the Stories

In [3]:
import re

In [3]:
def text_preprocessing(input_text):
  text= input_text.lower() #all the words are converted to lowercase
  text =  ''.join([i for i in text if i not in ''',.<>/?@#$%^&*_~!()-'"\[]{};:''']) #All the punctuation is removed
  text = "".join(filter(lambda x: not x.isdigit(), text)) #No numerical token is present
  text = " ".join([i for i in text.split() if i !='cnn']) #you would want to strip the word “cnn”..
  text= text.strip() #strip the final text
  return text

In [4]:
for k, v in master_dict.items():
    v[0] = text_preprocessing(str(v[0]))
    for i in range(len(v[1])):
        v[1][i] = text_preprocessing(str(v[1][i]))

In [5]:
#for testing purposes - dummy example
for k, v in master_dict.items():
    if k=='0001d1afc246a7964130f43ae940af6bc6c57f01.story':
        print(k, v)
        print(v[0])
        print(v[1])

0001d1afc246a7964130f43ae940af6bc6c57f01.story ['it s official us president barack obama wants lawmakers to weigh in on whether to use military force in syria obama sent a letter to the heads of the house and senate on saturday night hours after announcing that he believes military action against syrian targets is the right step to take over the alleged use of chemical weapons the proposed legislation from obama asks congress to approve the use of military force to deter disrupt prevent and degrade the potential for future uses of chemical weapons or other weapons of mass destruction it s a step that is set to turn an international crisis into a fierce domestic political battle there are key questions looming over the debate what did un weapons inspectors find in syria what happens if congress votes no and how will the syrian government react in a televised address from the white house rose garden earlier saturday the president said he would take his case to congress not because he has

In [ ]:
!pip install rouge

In [6]:
from rouge import Rouge
rouge = Rouge()

In [7]:
def get_rouge_scores(hypothesis, reference ):
    rouge_scores = rouge.get_scores(hypothesis, reference )
    return list(map(lambda x : (x['rouge-1']['f'],
                                x['rouge-2']['f']),
                    rouge_scores)
                )[0]

In [9]:
#for testing purposes - dummy example
print(get_rouge_scores(
    'it s official us president barack obama wants lawmakers to weigh in on whether',
'syrian official obama climbed to the top of the tree doesn t know how to get down'
))

(0.19354838214360054, 0.0)


In [15]:
final_doc=[]
import numpy as np
def calculate_rouge(story_key,story_list_txt, highlights_list,l):
    rouge_score1 = []
    rouge_score2 = []
    if len(story_list_txt)==0:
        rouge_score1.append([story_key,l,'','','',0])
        rouge_score2.append([story_key,l,'','','',0])
        final_doc.append([story_key,l,'','','',0])
    else:
        for i, j in enumerate([story_list_txt]):
            for n,h in enumerate(highlights_list):
                if len(h)==0 :
                    rouge_score1.append([story_key,l,j,'',0])
                    rouge_score2.append([story_key,l,j,'',0])
                    final_doc.append([story_key,l,j,'',0])
                else:
                    scores = get_rouge_scores(j,h)
                    rouge_score1.append([story_key,l,j, h, scores[0]])
                    rouge_score2.append([story_key,l, j, h, scores[1]])
                    final_doc.append([story_key,l, j, n,h, scores[1]])
    return np.array(rouge_score1), np.array(rouge_score2)


In [16]:
#dummy testing
l = 0
final_doc = []
for k, v in master_dict.items():
    if k in ('0001d1afc246a7964130f43ae940af6bc6c57f01.story',
             '000e009f6b1d954d827c9a550f3f24a5474ee82b.story'):
        roug1,roug2 = calculate_rouge(k,v[0],v[1],l)
        l+=1
for i in final_doc:
    print(i)

[[['0001d1afc246a7964130f43ae940af6bc6c57f01.story', 0, 'it s official us president barack obama wants lawmakers to weigh in on whether to use military force in syria obama sent a letter to the heads of the house and senate on saturday night hours after announcing that he believes military action against syrian targets is the right step to take over the alleged use of chemical weapons the proposed legislation from obama asks congress to approve the use of military force to deter disrupt prevent and degrade the potential for future uses of chemical weapons or other weapons of mass destruction it s a step that is set to turn an international crisis into a fierce domestic political battle there are key questions looming over the debate what did un weapons inspectors find in syria what happens if congress votes no and how will the syrian government react in a televised address from the white house rose garden earlier saturday the president said he would take his case to congress not becaus

In [ ]:
l = 0
final_doc = []
for k, v in master_dict.items():
    roug1,roug2 = calculate_rouge(k,v[0],v[1],l)
    l+=1

In [19]:
#dummy testing
for i,j in enumerate(final_doc):
    print(j)
    if i > 50:
        break

['0001d1afc246a7964130f43ae940af6bc6c57f01.story', 0, 'it s official us president barack obama wants lawmakers to weigh in on whether to use military force in syria obama sent a letter to the heads of the house and senate on saturday night hours after announcing that he believes military action against syrian targets is the right step to take over the alleged use of chemical weapons the proposed legislation from obama asks congress to approve the use of military force to deter disrupt prevent and degrade the potential for future uses of chemical weapons or other weapons of mass destruction it s a step that is set to turn an international crisis into a fierce domestic political battle there are key questions looming over the debate what did un weapons inspectors find in syria what happens if congress votes no and how will the syrian government react in a televised address from the white house rose garden earlier saturday the president said he would take his case to congress not because 

In [20]:
import pandas as pd
df = pd.DataFrame(final_doc)

In [21]:
df.head(5)

,0,1,2,3,4,5
0,0001d1afc246a7964130f43ae940af6bc6c57f01.story,0,it s official us president barack obama wants ...,0,syrian official obama climbed to the top of th...,0.014175
1,0001d1afc246a7964130f43ae940af6bc6c57f01.story,0,it s official us president barack obama wants ...,1,obama sends a letter to the heads of the house...,0.011635
2,0001d1afc246a7964130f43ae940af6bc6c57f01.story,0,it s official us president barack obama wants ...,2,obama to seek congressional approval on milita...,0.003883
3,0001d1afc246a7964130f43ae940af6bc6c57f01.story,0,it s official us president barack obama wants ...,3,aim is to determine whether cw were used not b...,0.007747
4,0002095e55fcbd3a2f366d9bf92a95433dc305ef.story,1,usain bolt rounded off the world championships...,0,usain bolt wins third gold of world championship,0.007547


In [25]:
df.columns=['story_file_title','story_id', 'story_text', 'summary_id', 'summary_text', 'rouge_score']
group_df = df.groupby('story_id').apply(lambda x: x.sort_values(["rouge_score"], ascending = False)).reset_index(drop=True)
final_df = group_df.groupby(["story_id"]).head(5)
final_df['rouge_score'] = final_df['rouge_score'].astype(np.float)
final_df['label'] = np.where(final_df['rouge_score']>0,1,0)
final_df

,story_file_title,story_id,story_text,summary_id,summary_text,rouge_score,label
0,0001d1afc246a7964130f43ae940af6bc6c57f01.story,0,it s official us president barack obama wants ...,0,syrian official obama climbed to the top of th...,0.014175,1
1,0001d1afc246a7964130f43ae940af6bc6c57f01.story,0,it s official us president barack obama wants ...,1,obama sends a letter to the heads of the house...,0.011635,1
2,0001d1afc246a7964130f43ae940af6bc6c57f01.story,0,it s official us president barack obama wants ...,3,aim is to determine whether cw were used not b...,0.007747,1
3,0001d1afc246a7964130f43ae940af6bc6c57f01.story,0,it s official us president barack obama wants ...,2,obama to seek congressional approval on milita...,0.003883,1
4,0002095e55fcbd3a2f366d9bf92a95433dc305ef.story,1,usain bolt rounded off the world championships...,3,jamaica double up in women s xm relay,0.011321,1
...,...,...,...,...,...,...,...
329173,ffff2dc1cc4888253a4733f808959f0b4eab26a6.story,92577,realsimplecom tired of counting sheep try one ...,1,only take overthecounter sleeping pills sparin...,0.013089,1
329174,ffff522cebe5ad9dcfb6dfc476b8f423f3f8dd34.story,92578,student news september download pdf maps relat...,2,witness a day in the life of chilean miners as...,0.007449,1
329175,ffff522cebe5ad9dcfb6dfc476b8f423f3f8dd34.story,92578,student news september download pdf maps relat...,0,find out how the un general assembly can impac...,0.003734,1
329176,ffff522cebe5ad9dcfb6dfc476b8f423f3f8dd34.story,92578,student news september download pdf maps relat...,1,consider what some cubans can expect as they m...,0.002486,1


In [31]:
master_data = df.iloc[:,:].\
              drop_duplicates().\
              merge(final_df[['story_id','summary_id','label']],
              on=['story_id','summary_id'],how='left').\
              drop_duplicates(['story_id','summary_id','summary_text'])
master_data['label'].fillna(0,inplace=True)
master_data

,story_file_title,story_id,story_text,summary_id,summary_text,rouge_score,label
0,0001d1afc246a7964130f43ae940af6bc6c57f01.story,0,it s official us president barack obama wants ...,0,syrian official obama climbed to the top of th...,0.014175,1
1,0001d1afc246a7964130f43ae940af6bc6c57f01.story,0,it s official us president barack obama wants ...,1,obama sends a letter to the heads of the house...,0.011635,1
2,0001d1afc246a7964130f43ae940af6bc6c57f01.story,0,it s official us president barack obama wants ...,2,obama to seek congressional approval on milita...,0.003883,1
3,0001d1afc246a7964130f43ae940af6bc6c57f01.story,0,it s official us president barack obama wants ...,3,aim is to determine whether cw were used not b...,0.007747,1
4,0002095e55fcbd3a2f366d9bf92a95433dc305ef.story,1,usain bolt rounded off the world championships...,0,usain bolt wins third gold of world championship,0.007547,1
...,...,...,...,...,...,...,...
329173,ffff2dc1cc4888253a4733f808959f0b4eab26a6.story,92577,realsimplecom tired of counting sheep try one ...,2,valerian tea when taken for less than two week...,0.020806,1
329174,ffff522cebe5ad9dcfb6dfc476b8f423f3f8dd34.story,92578,student news september download pdf maps relat...,0,find out how the un general assembly can impac...,0.003734,1
329175,ffff522cebe5ad9dcfb6dfc476b8f423f3f8dd34.story,92578,student news september download pdf maps relat...,1,consider what some cubans can expect as they m...,0.002486,1
329176,ffff522cebe5ad9dcfb6dfc476b8f423f3f8dd34.story,92578,student news september download pdf maps relat...,2,witness a day in the life of chilean miners as...,0.007449,1


In [32]:
#serialization
# file has been saved on client os
import pickle
with open('master_data.pkl', 'wb') as f:
    pickle.dump(master_data, f)




















